In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
import tarfile
import gdown
from gslTranslater.constants import *
from gslTranslater.utils.common import read_yaml, create_directories, get_size
from gslTranslater import logger

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Dev\\Upwork\\GSL\\GSL-Project'

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.tar_dir,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            tar_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {tar_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, tar_download_dir, quiet=False)

            logger.info(f"Downloaded data from {dataset_url} into file {tar_download_dir}")

        except Exception as e:
            raise e

    def extract_tar_file(self):
        extract_path = self.config.unzip_dir
        os.makedirs(extract_path, exist_ok=True)
        with tarfile.open(self.config.local_data_file, 'r:gz') as tar_ref:
            tar_ref.extractall(extract_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_tar_file()
except Exception as e:
    raise e

[2024-08-06 13:25:08,897: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-06 13:25:08,964: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-06 13:25:08,965: INFO: common: created directory at: artifacts]
[2024-08-06 13:25:08,966: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-06 13:25:08,968: INFO: 1579108944: Downloading data from https://drive.google.com/file/d/1WILXp60jYb70wby_vBp-giuzLUaZt-h-/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1WILXp60jYb70wby_vBp-giuzLUaZt-h-
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1WILXp60jYb70wby_vBp-giuzLUaZt-h-&confirm=t&uuid=6ee27726-4bf9-4243-95a7-93b4cd651a39
To: d:\Dev\Upwork\GSL\GSL-Project\artifacts\data_ingestion\data.zip
100%|██████████| 170M/170M [02:14<00:00, 1.26MB/s] 

[2024-08-06 13:27:27,031: INFO: 1579108944: Downloaded data from https://drive.google.com/file/d/1WILXp60jYb70wby_vBp-giuzLUaZt-h-/view?usp=sharing into file artifacts/data_ingestion/data.zip]
